In [1]:
import itertools
gateway_combination = [1,1,1]
priority_orders = list(itertools.permutations([i for i, y in enumerate(gateway_combination) if y == 1]))
# priority_orders = list(itertools.permutations(range(4)))
priority_orders
# I need priority values as: [0,1,2] and [0,2,1]

[(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)]

# Full Brute force code

In [3]:
import itertools
import copy

def minimize_power(nodes, gateways, links, debug=True):
    budgets = [i['budget'] for i in nodes]
    data = [i['data'] for i in nodes]

    min_power = float('inf')
    min_configuration = None
    # Generate all possible combinations of gateway setups
    for node_combination in itertools.permutations(range(len(nodes))):
        for gateway_combination in itertools.product([0, 1], repeat=len(gateways)):
            if debug: print("\n\n@@@@@@@@@@@@@@@@@@@@@@\ngateway combination: ", list(gateway_combination))
            total_gateway_power = 0
            
            # Calculate total power for the current gateway combination
            for i, is_gateway_on in enumerate(gateway_combination):
                if debug: print("\ti: ",i,"; Gateway on/off: ",is_gateway_on,end="; ")
                if is_gateway_on:
                    if debug: print("\t\tPower: ", gateways[i]['power'])
                    total_gateway_power += gateways[i]['power']
                if debug: print()
            if debug: print("Total Power to set up gateways: ", total_gateway_power)
            # If there are multiple gateways turned on, create priority order
            gateway_priority_orders = itertools.permutations([i for i, val in enumerate(gateway_combination) if val])
            # if debug: print("gateway_priority_orders: ", gateway_priority_orders)

            each_config_link_power = 0
            # Iterate over priority orders
            for g_priority_order in gateway_priority_orders:
                each_config_link_power = total_gateway_power
                if debug: print("\n\tcurr priority order: ", list(g_priority_order))
                node_data_remaining = copy.deepcopy(data)
                node_budget_remaining = copy.deepcopy(budgets)
                gateways_copy = copy.deepcopy(gateways)
                satisfied_nodes = [0] * len(nodes)  # Track which nodes are satisfied by each gateway
                # Assign data to gateways based on priority order
                for node_idx in node_combination:
                    remaining_data = node_data_remaining[node_idx]

                    for gateway_idx in g_priority_order:
                        if remaining_data <= 0:
                            break

                        if debug: print("\t\t\tNode: " ,node_idx,"; budget: ",nodes[node_idx], "; In gateway: ", gateway_idx, "; Remaining_data: ", remaining_data, "; Gateway: " ,gateways_copy[gateway_idx])
                        
                        if not gateways_copy[gateway_idx]: 
                            if debug: print("\t\t\t\tGateway not setup!")
                            continue
                        link_power = links[node_idx][gateway_idx]
                        if debug: print("\t\t\t\t\t\tLink Power: ", link_power)
                        if link_power<=0: 
                            if debug: print("\t\t\t\t\tNo Link!")
                            continue
                        if nodes[node_idx]['budget'] >= link_power and gateways_copy[gateway_idx]['storage']>0: 
                            # If battery left in node and gateway has storage.
                            if remaining_data>=gateways_copy[gateway_idx]['storage']: 
                                remaining_data -= gateways_copy[gateway_idx]['storage'] 
                                node_budget_remaining[node_idx]-=link_power
                                each_config_link_power+=link_power
                                gateways_copy[gateway_idx]['storage'] = 0
                                if debug: print("\t\t\t\t\t\t\tGateway storage is NULL!")
                                if debug: print("Remaining Data: ", remaining_data)
                                if remaining_data==0:
                                    satisfied_nodes[node_idx]=1
                                    if debug: print("\t\t\t\t\t\t\tNode satisfied!!")
                            else:
                                satisfied_nodes[node_idx] = 1
                                each_config_link_power+=link_power
                                gateways_copy[gateway_idx]['storage'] -= remaining_data
                                if debug: print("\t\t\t\t\t\t\tGateway storage is NOT Null: ",gateways_copy[gateway_idx]['storage'] )
                                if debug: print("\t\t\t\t\t\tNode satisfied!!")
                                break
                        if debug: print("Satisfied Nodes: ",satisfied_nodes)

                # Check if all nodes are satisfied
                if all(satisfied_nodes):
                    if debug: print("\t\tAll nodes are satisfied!")
                    if each_config_link_power < min_power:
                        min_power = each_config_link_power
                        min_configuration = (gateway_combination, satisfied_nodes)
                        if debug: print("\t\t\tGLOBAL Min Power is UPDATED!!")
                if debug: print("\t> Total Configuration Power: ", each_config_link_power)
    return min_power, min_configuration

# Example input data
nodes = [{'budget': 8,'data':5}, {'budget': 10,'data':8}, {'budget': 12,'data':10}]
gateways = [{'power': 10, 'storage': 23}, {'power': 15, 'storage': 25}]  # Gateway setup power and storage
links = [[2,3], [4,5], [6,7]]  # Transmission power from node i to gateway j

# Run the algorithm
min_power, min_configuration = minimize_power(nodes, gateways, links)
if min_power==float('inf'):
    print("All nodes are not satisfied!!")
else:
    print("Minimum power:", min_power)
print("Gateway configuration:", min_configuration)




@@@@@@@@@@@@@@@@@@@@@@
gateway combination:  [0, 0]
	i:  0 ; Gateway on/off:  0; 
	i:  1 ; Gateway on/off:  0; 
Total Power to set up gateways:  0

	curr priority order:  []
	> Total Configuration Power:  0


@@@@@@@@@@@@@@@@@@@@@@
gateway combination:  [0, 1]
	i:  0 ; Gateway on/off:  0; 
	i:  1 ; Gateway on/off:  1; 		Power:  15

Total Power to set up gateways:  15

	curr priority order:  [1]
			Node:  0 ; budget:  {'budget': 8, 'data': 5} ; In gateway:  1 ; Remaining_data:  5 ; Gateway:  {'power': 15, 'storage': 25}
						Link Power:  3
							Gateway storage is NOT Null:  20
						Node satisfied!!
			Node:  1 ; budget:  {'budget': 10, 'data': 8} ; In gateway:  1 ; Remaining_data:  8 ; Gateway:  {'power': 15, 'storage': 20}
						Link Power:  5
							Gateway storage is NOT Null:  12
						Node satisfied!!
			Node:  2 ; budget:  {'budget': 12, 'data': 10} ; In gateway:  1 ; Remaining_data:  10 ; Gateway:  {'power': 15, 'storage': 12}
						Link Power:  7
							Gateway storage is 

In [ ]:
import itertools
import copy

def minimize_power(nodes, gateways, links, debug=True):
    budgets = [i['budget'] for i in nodes]
    data = [i['data'] for i in nodes]

    min_power = float('inf')
    min_configuration = None
    # Generate all possible combinations of gateway setups
    for node_combination in itertools.permutations(range(len(nodes))):
        for gateway_combination in itertools.product([0, 1], repeat=len(gateways)):
            if debug: print("\n\n@@@@@@@@@@@@@@@@@@@@@@\ngateway combination: ", list(gateway_combination))
            total_gateway_power = 0
            
            # Calculate total power for the current gateway combination
            for i, is_gateway_on in enumerate(gateway_combination):
                if debug: print("\ti: ",i,"; Gateway on/off: ",is_gateway_on,end="; ")
                if is_gateway_on:
                    if debug: print("\t\tPower: ", gateways[i]['power'])
                    total_gateway_power += gateways[i]['power']
                if debug: print()
            if debug: print("Total Power to set up gateways: ", total_gateway_power)
            # If there are multiple gateways turned on, create priority order
            gateway_priority_orders = itertools.permutations([i for i, val in enumerate(gateway_combination) if val])
            # if debug: print("gateway_priority_orders: ", gateway_priority_orders)

            each_config_link_power = 0
            # Iterate over priority orders
            for g_priority_order in gateway_priority_orders:
                each_config_link_power = total_gateway_power
                if debug: print("\n\tcurr priority order: ", list(g_priority_order))
                node_data_remaining = copy.deepcopy(data)
                node_budget_remaining = copy.deepcopy(budgets)
                gateways_copy = copy.deepcopy(gateways)
                satisfied_nodes = [0] * len(nodes)  # Track which nodes are satisfied by each gateway
                # Assign data to gateways based on priority order
                for node_idx in node_combination:
                    remaining_data = node_data_remaining[node_idx]

                    for gateway_idx in g_priority_order:
                        if remaining_data <= 0:
                            break

                        if debug: print("\t\t\tNode: " ,node_idx,"; budget: ",nodes[node_idx], "; In gateway: ", gateway_idx, "; Remaining_data: ", remaining_data, "; Gateway: " ,gateways_copy[gateway_idx])
                        
                        if not gateways_copy[gateway_idx]: 
                            if debug: print("\t\t\t\tGateway not setup!")
                            continue
                        link_power = links[node_idx][gateway_idx]
                        if debug: print("\t\t\t\t\t\tLink Power: ", link_power)
                        if link_power<=0: 
                            if debug: print("\t\t\t\t\tNo Link!")
                            continue
                        if nodes[node_idx]['budget'] >= link_power and gateways_copy[gateway_idx]['storage']>0: 
                            # If battery left in node and gateway has storage.
                            if remaining_data>=gateways_copy[gateway_idx]['storage']: 
                                remaining_data -= gateways_copy[gateway_idx]['storage'] 
                                node_budget_remaining[node_idx]-=link_power
                                each_config_link_power+=link_power
                                gateways_copy[gateway_idx]['storage'] = 0
                                if debug: print("\t\t\t\t\t\t\tGateway storage is NULL!")
                                if debug: print("Remaining Data: ", remaining_data)
                                if remaining_data==0:
                                    satisfied_nodes[node_idx]=1
                                    if debug: print("\t\t\t\t\t\t\tNode satisfied!!")
                            else:
                                satisfied_nodes[node_idx] = 1
                                each_config_link_power+=link_power
                                gateways_copy[gateway_idx]['storage'] -= remaining_data
                                if debug: print("\t\t\t\t\t\t\tGateway storage is NOT Null: ",gateways_copy[gateway_idx]['storage'] )
                                if debug: print("\t\t\t\t\t\tNode satisfied!!")
                                break
                        if debug: print("Satisfied Nodes: ",satisfied_nodes)

                # Check if all nodes are satisfied
                if all(satisfied_nodes):
                    if debug: print("\t\tAll nodes are satisfied!")
                    if each_config_link_power < min_power:
                        min_power = each_config_link_power
                        min_configuration = (gateway_combination, satisfied_nodes)
                        if debug: print("\t\t\tGLOBAL Min Power is UPDATED!!")
                if debug: print("\t> Total Configuration Power: ", each_config_link_power)
    return min_power, min_configuration

# Example input data
nodes = [{'budget': 8,'data':5}, {'budget': 10,'data':8}, {'budget': 12,'data':10}]
gateways = [{'power': 10, 'storage': 23}, {'power': 15, 'storage': 25}]  # Gateway setup power and storage
links = [[2,3], [4,5], [6,7]]  # Transmission power from node i to gateway j

# Run the algorithm
min_power, min_configuration = minimize_power(nodes, gateways, links)
if min_power==float('inf'):
    print("All nodes are not satisfied!!")
else:
    print("Minimum power:", min_power)
print("Gateway configuration:", min_configuration)
